In [8]:
import paramiko

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"

## server ##
server_ip = "172.16.16.122"
server_user = "root"
server_pw = "asd123!@"
server_iqn = "iqn.2025-11.local.r1:disk1"

## client ##
client_ip = "172.16.16.25"
client_user = "root"
client_pw = "asd123!@"
client_os_type = "R"
client_iqn = "iqn.1994-05.com.redhat:64d3173ff775"


def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def iscsi_server_install():
    print("install을 시작합니다.")
    cc("dnf update -y && dnf upgrade -y")
    cc("dnf install targetcli -y")
    cc("systemctl stop firewalld")
    cc("dnf install -y expect")

def iscsi_server_targetcli_start() :
    print("targetcli 시작합니다.")
    cc("systemctl enable --now target;")

def iscsi_server_setting():
    global server_iqn 
    print("iscsi server를 세팅해보겠습니다~")
    cc("lsblk")
    disk = input("위에 결과에서 확인한 공유할 물리 디스크를 입력해주세요 ex. /dev/sdb :")
    domain = input("iSCSI 도메인을 입력해주세요. ex. 2025-11.local.r1:disk1 :")
    server_iqn = f"iqn.{domain}"
    
    commands = f"""
cat << EOF > ~/target_server_setting.exp
#!/usr/bin/expect -f
spawn targetcli

expect "/> " 
send "backstores/block create name=disk1 dev={disk}\\r"

expect "/> "
send "iscsi/ create {server_iqn}\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/luns create /backstores/block/disk1\\r"

expect "/> "
send "exit\\r"
expect eof
EOF

chmod +x ~/target_server_setting.exp 
expect ~/target_server_setting.exp
"""
    cc(commands)


def change_remote_cc_from_server_to_client():
    global ip
    global user
    global pw
    ip = client_ip
    user = client_user
    pw = client_pw

def change_remote_cc_from_client_to_server():
    global ip
    global user
    global pw
    ip = server_ip
    user = server_user
    pw = server_pw

def iscsi_client_install_start() :
    global client_os_type
    global client_iqn

    change_remote_cc_from_server_to_client()
    
    client_os_type = input("client가 Rocky인 경우 R , Ubuntu인 경우 U를 입력해주세요")
    
    commands = "" 
    if(client_os_type == "R") :
        print("client : Rocky")
        commands = """
sudo dnf install -y iscsi-initiator-utils
sudo systemctl enable --now iscsid
cat /etc/iscsi/initiatorname.iscsi
        """
        #cc("ls -l")
        cc(commands)
        
        client_iqn = input("위에서 확인한 client의 iqn을 복사해주세요. ex. iqn.1994-05.com.redhat:30eb3450bd10 :")
        
#     if(client_os_type == "U") :
#         commands = """
# apt update && apt upgrade -y
# apt install targetcli-fb -y
# systemctl stop ufw
# systemctl enable --now target
# systemctl status target
#         """
    
def iscsi_server_set_clientsIQN_at_ACL():
    change_remote_cc_from_client_to_server()
    cmd = f"""
cat << EOF > ~/target_server_set_acl.exp
#!/usr/bin/expect -f
spawn targetcli

expect "/> "
send "iscsi/{server_iqn}/tpg1/acls create {client_iqn}\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/portals delete 0.0.0.0 3260\\r"

expect "/> "
send "iscsi/{server_iqn}/tpg1/portals create {server_ip}\\r"

expect "/> "
send "exit\\r"
expect eof
EOF

chmod +x ~/target_server_set_acl.exp
expect ~/target_server_set_acl.exp

systemctl enable target
systemctl restart target
systemctl status target
"""
    cc(cmd)

def iscsi_client_login_set_servers_iscsi():
    change_remote_cc_from_server_to_client()
    cc(f"iscsiadm -m discovery -t st -p {server_ip}")
    cc(f"iscsiadm -m node -T {server_iqn} -p {server_ip}  -l")
    
def iscsi() :
    iscsi_server_install()
    iscsi_server_targetcli_start()
    iscsi_server_setting()
    iscsi_client_install_start()
    iscsi_server_set_clientsIQN_at_ACL()
    iscsi_client_login_set_servers_iscsi()

iscsi()

172.16.16.122:3260,1 iqn.2025-11.local.r1:disk1

Logging in to [iface: default, target: iqn.2025-11.local.r1:disk1, portal: 172.16.16.122,3260]
Login to [iface: default, target: iqn.2025-11.local.r1:disk1, portal: 172.16.16.122,3260] successful.

